#Preparacion del entorno

In [ ]:
import os
from urllib.request import urlretrieve
from zipfile import ZipFile
import shutil

github_url = "https://github.com/SalvaMrS/TP2_NLP/archive/main.zip"
zip_file_path, _ = urlretrieve(github_url, "TP2_NLP.zip")

with ZipFile(zip_file_path, "r") as zip_ref:
    zip_ref.extractall("/content")

os.remove(zip_file_path)

source_dir = "/content/TP2_NLP-main/Documentos"
target_dir = "/content/Documentos"

os.makedirs(target_dir, exist_ok=True)

for filename in os.listdir(source_dir):
    source_path = os.path.join(source_dir, filename)
    target_path = os.path.join(target_dir, filename)
    shutil.move(source_path, target_path)

shutil.rmtree('/content/TP2_NLP-main')

os.listdir("/content")

['.config', 'Documentos', 'sample_data']

#Creacion del df

In [ ]:
!pip install PyPDF2

In [ ]:
import PyPDF2

def leer_texto(texto):
  with open(texto, 'rb') as archivo:
      lector = PyPDF2.PdfReader(archivo)
      text = ''
      for i in range(len(lector.pages)):
          pagina = lector.pages[i]
          text += pagina.extract_text()
  return text

In [ ]:
import re

def segmentar_texto(texto, doc):
  expresion = r'A[Rr][Tt].{0,8}\d{1,4}.{0,3}[-—.]'
  expresion_regular = re.compile(expresion, re.MULTILINE)

  resultado = re.split(expresion_regular, texto)
  machs = expresion_regular.findall(texto)

  resultado.pop(0)

  expresion_resultado = re.compile(r'^.*?(?=\.\n)')
  resultado_final = [expresion_resultado.search(item).group() if expresion_resultado.search(item) else item for item in resultado]

  resultado_final = []

  for i, tex in enumerate(resultado):
      texto_formateado = tex.replace('\n', ' ')
      resultado_final.append((machs[i], doc + ': ' + machs[i] + texto_formateado))

  return resultado_final

In [ ]:
import pandas as pd
import re

def obtener_numero_articulo(str_articulo):
    numero_articulo = re.search(r'\d{1,4}', str_articulo)
    return int(numero_articulo.group()) if numero_articulo else None

def procesar_archivo(archivo_path):
    texto = leer_texto(archivo_path)

    nombre_doc = archivo_path.split("/")[-1].split("-")[0]

    resultado_segmentacion = segmentar_texto(texto, nombre_doc)

    data = pd.DataFrame(resultado_segmentacion, columns=['articulo', 'texto'])
    data['doc'] = nombre_doc

    data['numero_articulo'] = data['articulo'].apply(obtener_numero_articulo)

    data = data[['doc', 'numero_articulo', 'articulo', 'texto']]

    return data

In [ ]:
import os

dir_data = "/content/Documentos"

list_data = [os.path.join(dir_data, filename)
  for filename in os.listdir(dir_data)
  if not filename.startswith('.')
  and filename.lower().endswith('.pdf')
  ]

data = pd.DataFrame(columns=['doc', 'numero_articulo', 'articulo', 'texto'])

for archivo_path in list_data:
  df_temp = procesar_archivo(archivo_path)
  data = pd.concat([data, df_temp], ignore_index=True)

#ChromaDB

In [ ]:
lista_texto = data['texto'].tolist()
lista_metadata = data[['doc', 'articulo', 'numero_articulo']].to_dict(orient='records')
lista_indices = list(map(str, list(data.index.tolist())))

In [ ]:
!pip install typing-extensions==3.10.0.2
!pip install chromadb -q

import chromadb

  Using cached typing_extensions-3.10.0.2-py3-none-any.whl (26 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
sqlalchemy 2.0.23 requires typing-extensions>=4.2.0, but you have typing-extensions 3.10.0.2 which is incompatible.
arviz 0.15.1 requires typing-extensions>=4.1.0, but you have typing-extensions 3.10.0.2 which is incompatible.
asgiref 3.7.2 requires typing-extensions>=4; python_version < "3.11", but you have typing-extensions 3.10.0.2 which is incompatible.
chex 0.1.7 requires typing-extensions>=4.2.0; python_version < "3.11", but you have t

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
tensorflow 2.14.0 requires numpy>=1.23.5, but you have numpy 1.23.4 which is incompatible.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [ ]:
chroma_client = chromadb.Client()
if len(chroma_client.list_collections()) != 0:
  chroma_client.delete_collection(name='Documentacion_de_leyes_Argentinas')
collection_ley = chroma_client.create_collection(name='Documentacion_de_leyes_Argentinas')

In [ ]:
collection_ley.add(
    documents=lista_texto,
    metadatas=lista_metadata,
    ids=lista_indices
)

#LLM

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.1 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
    Using cached setuptools-69.0.2-py3-none-any.whl (819 kB)
    Using cached scikit_build-0.17.6-py3-none-any.whl (84 kB)
    Using cached cmake-3.28.0-py2.py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (26.3 MB)
    Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl (307 kB)
    Using cached distro-1.8.0-py3-none-any.whl (20 kB)
    Using cached packaging-23.2-py3-none-any.whl (53 kB)
    Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
    Using cached wheel-0.42.0-py3-none-any.whl (65 kB)
    Creating /tmp/pip-build-env-ye2bjx4r/overlay/local/bin
    changing mode of /tmp/pip-build-env-ye2bjx4r/overlay/local/bin/ninja to 755
    changing mode

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [ ]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,
    n_batch=512,
    n_gpu_layers=128,
    n_ctx=2048
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
def consulta_modelo(consulta, texto_articulos, Descripcion, respuesta_anterior):
  prompt_template = f'''Rol: {Descripcion}

  Articulos similares: {texto_articulos}

  Usuario: {consulta}

  {respuesta_anterior}'''


  response = lcpp_llm(prompt=prompt_template, max_tokens=2048, temperature=0.8, top_p=0.95,
                    repeat_penalty=1.2, top_k=150)

  return response["choices"][0]["text"]

In [ ]:
def consulta_legal(consulta):
  articulos = collection_ley.query(
      query_texts=[consulta],
      n_results=5
  )

  texto_articulos = '\nArticulos similares: '.join(articulos['documents'][0])
  Descripcion = "Eres un consultor de leyes Argentinas que dada la consulta del Usurario y según la documentacion da la respuesta mas detellada y completa posible y en español, siempre responde con lo que dice los articulos de la documentacion de manera adaptada a personas sin conocimientos legales y nunca hace preguntas. tu funcion es responder solo lo que dice el Consultor y nada más."
  introduccion = 'Consultor: Buenos días/tardes Usuario(a), como consultor de leyes Argentinas, puedo informarte que'

  respuesta_procesada = consulta_modelo(consulta, texto_articulos, Descripcion, introduccion)

  return introduccion + respuesta_procesada

In [ ]:
while True:
    print('''

----------------------------------------------
''')

    try:
        consulta = str(input("Ingrese su consulta legal('q' para salir): "))
        if consulta.lower() == 'q':
          print()
          print('Saliendo...')
          break
    except Exception as e:
        print(f"Error al leer la entrada del usuario: {e}")
        continue

    resultado = consulta_legal(consulta)

    lineas_resultado = resultado.split('\n')
    longitud_maxima = 100
    resultado_segmentado = [linea[i:i+longitud_maxima] for linea in lineas_resultado for i in range(0, len(linea), longitud_maxima)]

    for linea in resultado_segmentado:
        print(linea)



----------------------------------------------

Ingrese su consulta legal('q' para salir): Q
